In [ ]:
# 初始化库
from canlib import canlib, Frame
import struct

import time

In [ ]:
# 枚举CAN端口
num_channels = canlib.getNumberOfChannels()
print(f"Found {num_channels} channels")
for ch in range(num_channels):
    chd = canlib.ChannelData(ch)
    print(f"{ch}. {chd.channel_name} ({chd.card_upc_no} / {chd.card_serial_no})")

#比特率
cdbBitrate=canlib.Bitrate.BITRATE_1M

In [ ]:
# 建立变量字典
varDict={}

tiMapFileLoc = """C:/Users/t/workspace_v12/P21kw_v2_SiC_S/build_release/P21kw_v2_SiC_S.map"""

with open(tiMapFileLoc, 'r') as f:
  mapFileTxt = f.read()

startMapChaStr = """GLOBAL SYMBOLS: SORTED BY Symbol Address"""

mapFileTxt = mapFileTxt[mapFileTxt.find(startMapChaStr)+len(startMapChaStr):-1]

mapFileTxtLines=mapFileTxt.split('\n')

# 扔掉前面4行和后面3行
mapFileTxtLines=mapFileTxtLines[4:-2]

# 创建变量表字典
for metaTxt in mapFileTxtLines:
    metaTxtList=metaTxt.split()
    if(not metaTxtList[2].startswith("_")):
        continue
    if(metaTxtList[2][1:].startswith("_")):
        continue
    metaTxtList[2]=metaTxtList[2][1:]
    varAddr=int(metaTxtList[1],16)
    if(varAddr < 0xb000 or varAddr >= 0x1C000):
        continue
    # print(metaTxtList[2], hex(varAddr))
    varDict[metaTxtList[2]] = varAddr

# 人工标注非float型的变量
varTypeDict = {}
varTypeDict["CH1_cur_mode"] = "int16_t"
varTypeDict["CH2_cur_mode"] = "int16_t"
varTypeDict["speed_mode"] = "int16_t"
varTypeDict["channel_mode"] = "int16_t"
varTypeDict["CH1_angle_mode"] = "int16_t"
varTypeDict["CH2_angle_mode"] = "int16_t"
varTypeDict["thetaRDC_raw_offset_CH1"] = "uint16_t"
varTypeDict["thetaRDC_raw_offset_CH2"] = "uint16_t"
varTypeDict["thetaRDC_raw"] = "uint16_t"
varTypeDict["IProtectFlg_CH1"] = "int16_t"
varTypeDict["IProtectFlg_CH2"] = "int16_t"
varTypeDict["SPDProtectFlg"] = "int16_t"
varTypeDict["UdcProtectFlg_CH1"] = "int16_t"
varTypeDict["UdcProtectFlg_CH2"] = "int16_t"
varTypeDict["tempProtectFlg"] = "int16_t"
varTypeDict["isr_start_pwm_cnt"] = "uint16_t"
varTypeDict["isr_end_pwm_cnt"] = "uint16_t"
varTypeDict["targetWaveMode"] = "int16_t"
varTypeDict["CH1_Iv_raw"] = "int16_t"
varTypeDict["CH1_Iw_raw"] = "int16_t"
varTypeDict["CH1_Idc_raw"] = "int16_t"
varTypeDict["CH1_Iv_raw_offset"] = "int16_t"
varTypeDict["CH1_Iw_raw_offset"] = "int16_t"
varTypeDict["CH1_Idc_raw_offset"] = "int16_t"
varTypeDict["CH2_Iu_raw"] = "int16_t"
varTypeDict["CH2_Iv_raw"] = "int16_t"
varTypeDict["CH2_Idc_raw"] = "int16_t"
varTypeDict["CH2_Iu_raw_offset"] = "int16_t"
varTypeDict["CH2_Iv_raw_offset"] = "int16_t"
varTypeDict["CH2_Idc_raw_offset"] = "int16_t"
varTypeDict["ad2s1210ErrGPIO"] = "uint16_t"
varTypeDict["dbg_ad2s1210Err"] = "uint16_t"
varTypeDict["ad2s1210WaitCnt"] = "int16_t"
varTypeDict["ADRC_mode"] = "uint16_t"
varTypeDict["RDV_err_check"] = "function"
varTypeDict["adcOffset_init"] = "function"
varTypeDict["scd_Udc_set"] = "function"
varTypeDict["sram_init"] = "function"
varTypeDict["ad2s1210_ClaAccModeEn"] = "function"


In [ ]:
# 解析结构体, 暂时只对纯float结构体
stru2Dtab_PIctrl_struct = [
    ["kp", 0],
    ["ki", 2],
    ["kb", 4],
    ["max", 6],
    ["min", 8],
    ["integral", 10],
    ["ans", 12],
]

stru2Dtab_Trans_struct = [
    ["U", 0],
    ["V", 2],
    ["W", 4],
    ["al", 6],
    ["be", 8],
    ["d", 10],
    ["q", 12],
    ["abdq0", 14],
]

def addStructVarUtil(structName, struInfoTab):
    global varDict
    for varInStru in struInfoTab:
        varDict[structName+"."+varInStru[0]] = varDict[structName]+varInStru[1]

addStructVarUtil("UdcPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("omegaPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IdPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IqPI", stru2Dtab_PIctrl_struct)

addStructVarUtil("CH1_ifbk", stru2Dtab_Trans_struct)


In [ ]:
# 保证变量全局性？
ch = canlib.openChannel(
    channel=0,
    flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
    bitrate=cdbBitrate,
)
ch.setBusOutputControl(canlib.Driver.NORMAL)
ch.busOn()
ch.busOff()
ch.close()

# 读取变量的辅助函数

def readMCUVarUtil(namex):
    global ch

    if(namex in varTypeDict):
        # TODO 补充其他情况
        if(varTypeDict[namex]=="uint16_t"):
        
            ch = canlib.openChannel(
                channel=0,
                flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
                bitrate=cdbBitrate,
            )
            ch.setBusOutputControl(canlib.Driver.NORMAL)
            ch.busOn()

            txFrameData = struct.pack("<I", varDict[namex]) + struct.pack("<I", 0)
            txFrame = Frame(id_=6, data=txFrameData, dlc=8)
            ch.write(txFrame)
            ch.writeSync(timeout=200)

            try:
                frame = ch.read(timeout=200)
                if(frame.flags!=2 or frame.id!=5 or frame.dlc!=8):
                    print("收到错误帧!")
                else:
                    can_val = struct.unpack("<H", frame.data[4:6])[0]

            except canlib.CanNoMsg:
                print("no reply")
                can_val = None

            ch.busOff()
            ch.close()

            return can_val

        else:
        # TODO 补充其他情况
            print("暂不支持该类型!")
    else:
        ch = canlib.openChannel(
            channel=0,
            flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
            bitrate=cdbBitrate,
        )
        ch.setBusOutputControl(canlib.Driver.NORMAL)
        ch.busOn()

        txFrameData = struct.pack("<I", varDict[namex]) + struct.pack("<I", 2)
        txFrame = Frame(id_=6, data=txFrameData, dlc=8)
        ch.write(txFrame)
        ch.writeSync(timeout=200)

        try:
            frame = ch.read(timeout=200)
            if(frame.flags!=2 or frame.id!=5 or frame.dlc!=8):
                print("收到错误帧!")
            else:
                can_val = struct.unpack("<f", frame.data[4:8])[0]

        except canlib.CanNoMsg:
            print("no reply")
            can_val = None

        ch.busOff()
        ch.close()

        return can_val

def setMCUVarUtil(namex, val):
    global ch
    comSta=True

    if(namex in varTypeDict):
        if(varTypeDict[namex]=="uint16_t"):
        
            ch = canlib.openChannel(
                channel=0,
                flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
                bitrate=cdbBitrate,
            )
            ch.setBusOutputControl(canlib.Driver.NORMAL)
            ch.busOn()

            txFrameData = struct.pack("<I", varDict[namex]) + struct.pack("<H", val) + struct.pack("<H", 0)
            txFrame = Frame(id_=3, data=txFrameData, dlc=8)
            ch.write(txFrame)
            ch.writeSync(timeout=200)

            try:
                frame = ch.read(timeout=200)
                if(frame.flags!=2 or frame.id!=5 or frame.dlc!=8):
                    print("收到错误帧!")
                else:
                    can_val = struct.unpack("<H", frame.data[4:6])[0]
                    if(abs(val-can_val)/(abs(val)+1e-12)>0.01):
                        print("通信出错!")
                        comSta=False

            except canlib.CanNoMsg:
                print("no reply")

            ch.busOff()
            ch.close()

            return comSta

        else:
        # TODO 补充其他情况
            print("暂不支持该类型!")
    else:
        ch = canlib.openChannel(
            channel=0,
            flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
            bitrate=cdbBitrate,
        )
        ch.setBusOutputControl(canlib.Driver.NORMAL)
        ch.busOn()

        txFrameData = struct.pack("<I", varDict[namex]) + struct.pack("<f", val)
        txFrame = Frame(id_=4, data=txFrameData, dlc=8)
        ch.write(txFrame)
        ch.writeSync(timeout=200)

        try:
            frame = ch.read(timeout=200)
            if(frame.flags!=2 or frame.id!=5 or frame.dlc!=8):
                print("收到错误帧!")
            else:
                can_val = struct.unpack("<f", frame.data[4:8])[0]
                if(abs(val-can_val)/(abs(val)+1e-12)>0.01):
                    print("通信出错!")
                    comSta=False

        except canlib.CanNoMsg:
            print("no reply")

        ch.busOff()
        ch.close()

        return comSta

In [ ]:
# 特殊异常关端口
# Inactivate the CAN chip.
ch.busOff()

# Close the channel.
ch.close()

In [ ]:
# 修改角度标定零点值
thetaRDC_raw_offset=0

setMCUVarUtil("thetaRDC_raw_offset_CH1", thetaRDC_raw_offset)
setMCUVarUtil("thetaRDC_raw_offset_CH2", thetaRDC_raw_offset)

In [ ]:
# 修改转速控制器PI参数

omega_kp=0.866667
omega_ki=0.000029

omega_kp=0.1
omega_ki=0.000029

setMCUVarUtil("omegaPI.kp", omega_kp)
setMCUVarUtil("omegaPI.ki", omega_ki)
setMCUVarUtil("omegaPI.kb", omega_ki)

In [ ]:
# 集中修改电压控制器PI参数
Udc_kp=733
Udc_ki=2.44

Udc_kp=700
Udc_ki=2

setMCUVarUtil("UdcPI.kp", Udc_kp)
setMCUVarUtil("UdcPI.ki", Udc_ki)

In [ ]:
# 集中修改电流控制器PI参数
d_kp=0.56
d_ki=0.006
q_kp=1.76
q_ki=0.006

setMCUVarUtil("CH1_IdPI.kp", d_kp)
setMCUVarUtil("CH1_IdPI.ki", d_ki)
setMCUVarUtil("CH1_IqPI.kp", q_kp)
setMCUVarUtil("CH1_IqPI.ki", q_ki)

setMCUVarUtil("CH2_IdPI.kp", d_kp)
setMCUVarUtil("CH2_IdPI.ki", d_ki)
setMCUVarUtil("CH2_IqPI.kp", q_kp)
setMCUVarUtil("CH2_IqPI.ki", q_ki)

In [ ]:

print(readMCUVarUtil("CH1_IdPI.kp"))
